In [1]:
from chromadb.utils import embedding_functions
import chromadb
import autogen

from chromadb.utils import embedding_functions
default_ef = embedding_functions.DefaultEmbeddingFunction()



In [2]:
import os
from dotenv import load_dotenv
load_dotenv()


config_list_claude = [
    {
        # Choose your model name.
        "model": "claude-3-5-sonnet-20240620",
        # You need to provide your API key here.
        "api_key": os.getenv("ANTHROPIC_API_KEY"),
        "api_type": "anthropic",
    }
]

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\r", "\t"])

In [4]:
CHROMA_DB_PATH="./tmp/chromadb"
CHROMA_COLLECTION="autogen-docs-test"
from autogen.agentchat.contrib.vectordb.chromadb import ChromaVectorDB

chroma_client = chromadb.PersistentClient(path=CHROMA_DB_PATH)
collection = chroma_client.get_or_create_collection(name=CHROMA_COLLECTION)


vector_db = ChromaVectorDB(path=CHROMA_DB_PATH, embedding_function = default_ef)

In [5]:
from autogen.agentchat.contrib.retrieve_assistant_agent import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from tqdm import tqdm

import glob

llm_config = {
    "request_timeout": 600,
    "config_list": config_list_claude,
    "temperature": 0
}

assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config,
)

rag_agent = RetrieveUserProxyAgent(
    name="rag_agent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "qa",
        "docs_path": "./docs",
        "vector_db":vector_db,
        "chunk_token_size": 2000,
        "model": config_list_claude[0]["model"],
        "client": chromadb.PersistentClient(path="./tmp/chromadb"),
        "embedding_model": default_ef,
        "collection_name": CHROMA_COLLECTION,
#        "embedding_function": openai_embedding_function,
        "custom_text_split_function": text_splitter.split_text,
        "get_or_create": True,  # set to False if you don't want to reuse an existing collection, but you'll need to remove the collection manually
    },
    code_execution_config=False,  # set to False if you don't want to execute the code

)

In [6]:
customer_proxy_agent = autogen.ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [7]:
onboarding_personal_information_agent = autogen.ConversableAgent(
    name="onboarding_personal_information_agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's information.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config={
        "config_list": config_list_claude,
    },
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [8]:
coordinator_agent = autogen.ConversableAgent(
    name="coordinator_agent",
    system_message="You are a helpful assistant that can coordinate customer onboarding assessment and utilize any context information provided."
                    "You are the primary cordinator who will receive suggestions or advice from other agents (regulatory_agent, validation_agent)."
                    "YOUR FINAL RESPONSE MUST BE THE COMPLETE ASSESSMENT of the customer onboarding. When the assessment is complete and all perspectives are integrated, you can respond with TERMINATE.",
    code_execution_config=False,
    llm_config={
        "config_list": config_list_claude,
    },
    human_input_mode="NEVER",
)

In [9]:
regulatory_agent = autogen.ConversableAgent(
    name="regulatory_agent",
    system_message="You are a helpful assistant that can summarize and gather required onboarding requirements for a customer."
                    "You utilize any context information provided."
                    "You can also use other agents (rag_agent) to retrieve current information about customer onboarding.",
    code_execution_config=False,
    llm_config={
        "config_list": config_list_claude,
    },
    human_input_mode="NEVER",
)

In [10]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with your onboarding."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
]

In [11]:
assistant.reset()
rag_agent.initiate_chat(assistant, message=rag_agent.message_generator,problem="What should I ask the customer for onboarding?", n_results=2)

2024-12-15 10:14:44,281 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `autogen-docs-test`.


Trying to create collection.


2024-12-15 10:14:44,721 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 14 chunks.
Model claude-3-5-sonnet-20240620 not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['9c40b2d8', '2ae9d964']]
Adding content of doc 9c40b2d8 to context.


Model claude-3-5-sonnet-20240620 not found. Using cl100k_base encoding.


...

Model claude-3-5-sonnet-20240620 not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['9c40b2d8', '2ae9d964', 'c04ebd35', '4dda7268', '7d5e145e', 'cee2b5f7']]
Adding content of doc c04ebd35 to context.


Model claude-3-5-sonnet-20240620 not found. Using cl100k_base encoding.


Adding content of doc 4dda7268 to context.
rag_agent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: What should I ask the customer for onboarding?

Context is: client explanatory materials used 
by the investment firm. Annex 1 of 
the Guidance articulates key 
considerations and content which 
needs to be included in the client 
explanatory materials to ensure 
that the explanation of 
sustainability preferences is 
effective. 
 
Step 2: Identification of client sustainability preferences 
Short Assessment 
Q1. Do you have sustainability preferences for your investment? 
 
 a) Yes 
 b) Don’t know, need to hear more about it 
 c) No, only care about financial aspects of the investment 
 
The investment firm 

ChatResult(chat_id=None, chat_history=[{'content': "You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the\ncontext provided by the user.\nIf you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\nYou must give as short an answer as possible.\n\nUser's question is: What should I ask the customer for onboarding?\n\nContext is: client explanatory materials used \nby the investment firm. Annex 1 of \nthe Guidance articulates key \nconsiderations and content which \nneeds to be included in the client \nexplanatory materials to ensure \nthat the explanation of \nsustainability preferences is \neffective. \n \nStep 2: Identification of client sustainability preferences \nShort Assessment \nQ1. Do you have sustainability preferences for your investment? \n \n\uf0f0 a) Yes \n\uf0f0 b) Don’t know, need to hear more about it \n\uf0f0 c) No, only care about financial aspects of the investment \n 

In [12]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_personal_information_agent (to customer_proxy_agent):

Hello, I'm here to help you get started with your onboarding.Could you tell me your name and location?

--------------------------------------------------------------------------------
customer_proxy_agent (to onboarding_personal_information_agent):

i am patrick and i live in prangins

--------------------------------------------------------------------------------


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


onboarding_personal_information_agent (to customer_proxy_agent):

Thank you, Patrick. It's nice to meet you. Prangins is a lovely location in Switzerland.

Now, could you please provide me with your email address? This will help us set up your account and send you important information about our product.

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
customer_proxy_agent (to onboarding_personal_information_agent):



--------------------------------------------------------------------------------
